In [2]:
import torch
import numpy as np


In [16]:
x = torch.tensor(6.7)
y = torch.tensor(0)


**Calculate Loss Using Manual Operation**

In [17]:
w = torch.tensor(1.0)
b = torch.tensor(0.0)


In [18]:
def binary_cross_entropy(predction, target):
  epsilon = 1e-8
  predicton = torch.clamp(predction, epsilon, 1 - epsilon)
  return -(target * torch.log(predction + epsilon) + (1 - target) * torch.log(1 - predction + epsilon))



In [19]:
z= w * x + b
y_pred = torch.sigmoid(z)
loss = binary_cross_entropy(y_pred, y)
print(loss)

tensor(6.7012)


****Derivatives****

In [25]:
dloss_dy_pred =( y_pred - y )/ (y_pred * (1 - y_pred))

In [26]:
dy_pred_dz = y_pred * (1 - y_pred)

In [27]:

dz_dw = x
dz_db = 1

In [28]:
dl_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dl_db = dloss_dy_pred * dy_pred_dz * dz_db

In [29]:
print(dl_dw)
print(dl_db)

tensor(6.6918)
tensor(0.9988)


**Uisng Pytorch**

In [31]:
x = torch.tensor(6.7)
y = torch.tensor(0)

In [32]:
w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [33]:
z = w * x + b
y_pred = torch.sigmoid(z)

In [35]:
loss = binary_cross_entropy(y_pred, y)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

In [37]:
 loss.backward()

In [39]:
w.grad

tensor(6.6917)

In [40]:
b.grad

tensor(0.9988)

**For Multivaraible Inputs**

In [43]:
X = torch.tensor([1.0 , 2.0 , 3.0 ] , requires_grad=True)
y = (X**2).mean()

In [44]:
y

tensor(4.6667, grad_fn=<MeanBackward0>)

In [45]:
y.backward()

In [48]:
X.grad

tensor([0.6667, 1.3333, 2.0000])

In [1]:
import torch
import torch.nn as nn

# Define a CNN with Conv1 followed by MaxPooling
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv1(x)  # Convolution
        print("After Conv1:", x.shape)  # Expect (batch_size, 6, 28, 28)
        x = self.pool(x)   # Max Pooling
        print("After Pooling:", x.shape)  # Expect (batch_size, 6, 14, 14)
        return x

# Create an input image of size (batch_size=1, channels=3, height=32, width=32)
input_image = torch.randn(1, 3, 32, 32)

# Instantiate and apply the model
model = SimpleCNN()
output = model(input_image)


After Conv1: torch.Size([1, 6, 28, 28])
After Pooling: torch.Size([1, 6, 14, 14])


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # Convolutional Layer 1: 16 filters of size 5x5
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0)

        # Convolutional Layer 2: 32 filters of size 5x5
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)

        # Max Pooling Layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully Connected Layers
        self.fc3 = nn.Linear(in_features=5*5*16, out_features=120)  # FC3 Layer
        self.fc4 = nn.Linear(in_features=120, out_features=84)      # FC4 Layer
        self.fc_out = nn.Linear(in_features=84, out_features=10)    # Output Layer (Softmax)

    def forward(self, x):
        x = F.relu(self.conv1(x))   # Apply Conv1 + ReLU
        print("After Conv1:", x.shape)  # Expected (batch, 16, 28, 28)

        x = self.pool(x)            # Apply Max Pooling
        print("After Max Pooling 1:", x.shape)  # Expected (batch, 16, 14, 14)

        x = F.relu(self.conv2(x))   # Apply Conv2 + ReLU
        print("After Conv2:", x.shape)  # Expected (batch, 32, 10, 10)

        x = self.pool(x)            # Apply Max Pooling
        print("After Max Pooling 2:", x.shape)  # Expected (batch, 32, 5, 5)

        x = torch.flatten(x, start_dim=1)  # Flatten the tensor
        print("After Flatten:", x.shape)  # Expected (batch, 800)

        x = F.relu(self.fc3(x))  # Fully Connected Layer 3
        print("After FC3:", x.shape)  # Expected (batch, 128)

        x = F.relu(self.fc4(x))  # Fully Connected Layer 4
        print("After FC4:", x.shape)  # Expected (batch, 64)

        x = self.fc_out(x)  # Final Output Layer (No activation because we use softmax in loss)
        print("After Output Layer:", x.shape)  # Expected (batch, 10)

        return x

# Create a sample input tensor (Batch size=1, 3 color channels, 32x32 image)
input_image = torch.randn(1, 3, 32, 32)

# Initialize the model
model = CNN()

# Pass the input through the model
output = model(input_image)

# Apply Softmax activation
output = F.softmax(output, dim=1)

# Display final output
print("\nFinal Output (Softmax Probabilities):", output)


After Conv1: torch.Size([1, 6, 28, 28])
After Max Pooling 1: torch.Size([1, 6, 14, 14])
After Conv2: torch.Size([1, 16, 10, 10])
After Max Pooling 2: torch.Size([1, 16, 5, 5])
After Flatten: torch.Size([1, 400])
After FC3: torch.Size([1, 120])
After FC4: torch.Size([1, 84])
After Output Layer: torch.Size([1, 10])

Final Output (Softmax Probabilities): tensor([[0.0975, 0.0934, 0.0975, 0.0951, 0.0975, 0.1044, 0.0918, 0.1000, 0.1107,
         0.1120]], grad_fn=<SoftmaxBackward0>)


In [4]:
output.argmax(axis =-1)

tensor([9])